In [3]:
!pip install geopy

     |████████████████████████████████| 112 kB 2.7 MB/s eta 0:00:01


In [220]:
import geopy.distance

In [222]:
def gen_coordinates(topleft=(42.018708, -87.822461), botright=(41.644748, -87.524522), radius=200):
    '''
    Generates a list of coordinates corresponding to center points of non-intersecting circles.
    
    Inputs:
        topleft: tuple that contains the coordinates for the northwest corner of the rectangle.
        botright: tuple that contains the coordinates for the southeast corner of the rectangle.
        radius: interval at which the coordinates will be generated (in meters).
        
    Outputs:
        list of tuples containing coordinates partially covering the area of the rectangle. 
    '''
    
    #trying to find out what 0.000001 change in lat and lon corresponds to in meters
    
    #make an estimate by taking the average of meter change in the farthest 2 points of the rectangle
    
    compare_tl = (topleft[0] + 1/1000000, topleft[1])
    compare_br = (botright[0] + 1/1000000, botright[1])
    
    #geopy.distance.distance(c1, c2).m gives the distance between coordinates c1 and c2 in meters.
    
    long_unit = (geopy.distance.distance(compare_tl, topleft).m + 
                geopy.distance.distance(compare_br, botright).m) / 2
    
    compare_tl = (topleft[0], topleft[1] + 1/1000000)
    compare_br = (botright[0], botright[1] + 1/1000000)
    lat_unit = (geopy.distance.distance(compare_tl, topleft).m + 
                geopy.distance.distance(compare_br, botright).m) / 2
    
    h = radius / lat_unit / 1000000
    v = radius / long_unit / 1000000

    #h and v are how much we should change the lat and long values to move north/south and 
    #east/west by radius meters.
    
    lats = np.arange(min(topleft[0], botright[0]), max(topleft[0], botright[0]), h)
    longs = np.arange(min(topleft[1], botright[1]), max(topleft[1], botright[1]), v)
    
    coords = [(round(lat, 6), round(long, 6)) for lat in lats for long in longs]
    
    #fillers will be used to generate lats and longs that cover the areas between the circles
    
    filler_tl = (round(topleft[0] - h / 2, 6), round(topleft[1] + v / 2, 6))
    filler_br = (round(botright[0] + h / 2, 6), round(botright[1] - v / 2, 6))

    
    
    return(coords, filler_tl, filler_br)

In [223]:
def gen_coords_helper(topleft=(42.018708, -87.822461), botright=(41.644748, -87.524522), radius=200):
    '''
    Helper function that calls gen_coordinates two times and aggregates results to cover for the 
    areas between the initial set of circles.
    
    Inputs:
        topleft: tuple that contains the coordinates for the northwest corner of the rectangle.
            Default value is the coordinates for the northwest corner of Chicago. 
        botright: tuple that contains the coordinates for the southeast corner of the rectangle.
            Default value is the coordinates for the southeast corner of Chicago. 
        radius: interval at which the coordinates will be generated (in meters).
        
    Outputs:
        list of tuples containing coordinates covering the area of the rectangle. 
    '''
    coords, filler_tl, filler_br = gen_coordinates(topleft, botright, radius)
    #add the coordinates for the areas in between the initial circles
    coords += gen_coordinates(filler_tl, filler_br, radius)[0]
    return coords

In [226]:
coordinates = gen_coords_helper(radius=500)

In [228]:
len(coordinates)

8313

In [229]:
coordinates[:10]

[(41.644748, -87.822461),
 (41.644748, -87.817959),
 (41.644748, -87.813458),
 (41.644748, -87.808956),
 (41.644748, -87.804454),
 (41.644748, -87.799953),
 (41.644748, -87.795451),
 (41.644748, -87.790949),
 (41.644748, -87.786448),
 (41.644748, -87.781946)]